In [2]:
import pandas as pd 
import pandas_ta as ta 
import yfinance as yf 
import MetaTrader5 as mt5
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.io as pio

In [44]:
def cleaning(df):
    df = df.copy()  # Make a copy to avoid modifying the original DataFrame
    df.columns = [name[0].lower() for name in df.columns]  # Convert column names to lowercase
    df.index = df.index.rename('date')  # Rename index to 'date'
    df.index = pd.to_datetime(df.index).strftime('%Y-%m-%d %H:%M:%S')  # Format index as datetime string
    df = df.reset_index(drop=False)  # Reset index and assign back to df
    return df

In [100]:
def chart(df, fib_df=None):
    fig = go.Figure(data=[go.Candlestick(
        x=df['date'],
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close']
    )])

    fig.update_layout(
        xaxis_rangeslider_visible=False
    )
    print('successfull candlestick plot')

    if fib_df is not None and not fib_df.empty:
        print('Fibonacci dataframe was given and its not empty')

        fib_df = fib_df.iloc[:-1] if fib_df.iloc[-1].isna().any() else fib_df
        
        # Add yellow boxes (assumed example ranges)
        for _, row in fib_df.iterrows():
            fig.add_shape(
                type="rect",
                x0=row['date'],
                x1=row['next_day'],
                y0=row['fib 61.8'],
                y1=row['stop_loss'],
                fillcolor="yellow" if row['trend'] == -1 else 'red',
                opacity=0.3,
                line_width=0
            )
        print('fibonacci levels were ploted')

    # Show the chart in a new browser tab
    pio.show(fig, renderer='browser')


In [64]:
gold_daily = cleaning(yf.download('GC=F', period='60d', interval='1d'))

[*********************100%***********************]  1 of 1 completed


In [65]:
gold_15 = cleaning(yf.download('GC=F',period='8d', interval='15m'))

[*********************100%***********************]  1 of 1 completed


In [94]:
gold_1h = cleaning(yf.download('GC=F',period='60d', interval='1h'))

[*********************100%***********************]  1 of 1 completed


In [66]:
gold_daily.head()

,date,close,high,low,open,volume
0,2024-11-14 00:00:00,2568.199951,2576.199951,2554.199951,2555.100098,27
1,2024-11-15 00:00:00,2565.699951,2565.699951,2565.699951,2565.699951,20
2,2024-11-18 00:00:00,2610.600098,2610.699951,2589.399902,2591.699951,136
3,2024-11-19 00:00:00,2627.100098,2627.699951,2619.300049,2621.899902,87
4,2024-11-20 00:00:00,2648.199951,2648.199951,2638.500000,2638.500000,27


In [67]:
gold_15.head(3)

,date,close,high,low,open,volume
0,2025-01-17 00:00:00,2745.100098,2745.600098,2744.300049,2745.199951,199
1,2025-01-17 00:15:00,2743.600098,2745.699951,2743.399902,2745.100098,284
2,2025-01-17 00:30:00,2741.800049,2744.000000,2741.100098,2743.600098,1174


In [250]:
def get_daily_fibonacci(df, levels_only=False, stop_loss=20):
    
    # Add a trend column: 1 for uptrend, -1 for downtrend
    df['trend'] = df.apply(lambda row: 1 if row['close'] > row['open'] else -1, axis=1)

    # Create new columns and calculate Fibonacci levels for each row
    df['fib 23.6'] = df['high'] - ((df['high'] - df['low']) * 0.236)
    df['fib 38.2'] = df['high'] - ((df['high'] - df['low']) * 0.382)
    df['fib 61.8'] = df['high'] - ((df['high'] - df['low']) * 0.618)
    df['fib 100.0'] = df['low']  # 100% Fibonacci level corresponds to the low price

    # Calculate next day's index
    # Shift the 'date' column to create 'next_day'
    df['next_day'] = df['date'].shift(-1)
    print(df['next_day'].dtype)
    # Replace NaT with the next day's date
    df['next_day'] = df['next_day'].fillna(pd.to_datetime(df['date'].iloc[-1]) + pd.Timedelta(days=1))


    # Calculate stop_loss based on the trend and Fibonacci levels
    df['stop_loss'] = df.apply(
        lambda row: (row['fib 61.8'] - stop_loss) if row['trend'] == 1 else (row['fib 61.8'] + stop_loss), axis=1
    )

    # Return only Fibonacci levels and trend if levels_only=True
    if levels_only:
        return df[["fib 23.6", 'fib 38.2', 'fib 61.8', 'fib 100.0', 'trend', 'next_day', 'stop_loss']]
    else:
        return df


In [251]:
gold_daily_fib = get_daily_fibonacci(gold_daily)

object


In [252]:
gold_daily_fib.tail(3)

,date,close,high,low,open,volume,trend,fib 23.6,fib 38.2,fib 61.8,fib 100.0,next_day,stop_loss
46,2025-01-22 00:00:00,2767.600098,2768.800049,2753.500000,2754.500000,2079,1,2765.189237,2762.955430,2759.344619,2753.500000,2025-01-23 00:00:00,2739.344619
47,2025-01-23 00:00:00,2763.100098,2765.000000,2745.399902,2760.000000,2201,1,2760.374377,2757.512763,2752.887140,2745.399902,2025-01-24 00:00:00,2732.887140
48,2025-01-24 00:00:00,2777.399902,2794.800049,2761.000000,2761.800049,187347,1,2786.823237,2781.888430,2773.911619,2761.000000,2025-01-25 00:00:00,2753.911619


In [253]:
chart(gold_15, fib_df=gold_daily_fib)

successfull candlestick plot
Fibonacci dataframe was given and its not empty
fibonacci levels were ploted


In [254]:
chart(gold_1h, fib_df=gold_daily_fib)

successfull candlestick plot
Fibonacci dataframe was given and its not empty
fibonacci levels were ploted


# BackTesting.py

In [131]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG

C:\Users\USER\Desktop\Python\Lib\site-packages\backtesting\_plotting.py:50: UserWarning:

Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.



Loading BokehJS ...

In [258]:
class Fib_daily(Strategy):

    def init(self):
        # Access data series
        self.fib = self.data['fib 61.8']  # Ensure 'fib 61.8' is a valid column in your data
        # print('fibonacci', self.fib)

        self.close = self.data.Close
        # print('close data',self.data.Close)

        self.previous_close = [self.data.Close[i-1] for i in range(1, len(self.data.Close))]
        # print('previous', self.previous_close)

        self.stop_loss = self.data.stop_loss

    def next(self):
        # Check if the previous close was above the Fibonacci level and the current close is below it 
        if self.previous_close[0] >= self.fib and self.close <= self.fib:
            self.buy()  # Enter a long position
            self.position.stop_loss = self.stop_loss
            self.position.take_profit = self.fib + 30
            self.previous_close.pop(0)

        # Check if the previous close was below the Fibonacci level and the current close is above it
        elif self.previous_close[0] < self.fib and self.close >= self.fib:
            self.sell()  # Exit the position or go short
            self.position.stop_loss = self.stop_loss
            self.position.take_profit = self.fib + 30
            self.previous_close.pop(0)



In [259]:
gold_daily.head(3)

,date,close,high,low,open,volume,trend,fib 23.6,fib 38.2,fib 61.8,fib 100.0,next_day,stop_loss
0,2024-11-14 00:00:00,2568.199951,2576.199951,2554.199951,2555.100098,27,1,2571.007951,2567.795951,2562.603951,2554.199951,2024-11-15 00:00:00,2542.603951
1,2024-11-15 00:00:00,2565.699951,2565.699951,2565.699951,2565.699951,20,-1,2565.699951,2565.699951,2565.699951,2565.699951,2024-11-18 00:00:00,2585.699951
2,2024-11-18 00:00:00,2610.600098,2610.699951,2589.399902,2591.699951,136,1,2605.673140,2602.563333,2597.536521,2589.399902,2024-11-19 00:00:00,2577.536521


## Preparing data for backtesting

In [260]:
def prepare_for_backtesting(df):
    df = df.copy()
    df = df.rename(columns=dict(open='Open', high='High', low='Low', close='Close'))
    df = df.set_index('date')
    print(df.index.dtype)
    df.index = pd.DatetimeIndex(df.index)
    print(df.index.dtype)

    return df


In [261]:
data = prepare_for_backtesting(gold_daily)

object
datetime64[ns]


In [262]:
# Create a backtest instance
bt = Backtest(data, Fib_daily, cash=10000, commission=.002)

# Run the backtest
stats = bt.run()
print(stats)

Start                     2024-11-14 00:00:00
End                       2025-01-24 00:00:00
Duration                     71 days 00:00:00
Exposure Time [%]                    95.91837
Equity Final [$]                   9410.77915
Equity Peak [$]                       10000.0
Return [%]                           -5.89221
Buy & Hold Return [%]                 8.14578
Return (Ann.) [%]                   -26.82544
Volatility (Ann.) [%]                 9.45684
CAGR [%]                            -19.38991
Sharpe Ratio                         -2.83662
Sortino Ratio                        -3.06896
Calmar Ratio                          -4.5527
Max. Drawdown [%]                    -5.89221
Avg. Drawdown [%]                    -5.89221
Max. Drawdown Duration       70 days 00:00:00
Avg. Drawdown Duration       70 days 00:00:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                       -6.77682
Worst Trade [%]                   

In [263]:
bt.plot()

GridPlot(id='p1271', ...)

In [143]:
gold_15.set_index('date').index.dtype

dtype('O')